In [1]:
import pandas as pd
import torch
import fastai

# The data
We will start working with a sample from the IMDB sentiment classification dataset. The goal for now is to understand the dataset and prepare it for working with neural nets.


In [2]:
data_path = fastai.untar_data(fastai.URLs.IMDB_SAMPLE)
data_path.ls()


[PosixPath('/Users/dani/.fastai/data/imdb_sample/models'),
 PosixPath('/Users/dani/.fastai/data/imdb_sample/texts.csv')]

Let's load and see the data:

In [3]:
dataset = pd.read_csv(data_path/'texts.csv')
dataset.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


Each row corresponds to one example: the sentiment `label`, input `text` and a special field added by the fastai guys to split the dataset for training and validation.

# Preparing the data
Before feeding a neural net with text, we need to turn it into sequences of numbers.
This can be done in many ways: an integer per word, char, subword, etc. 
Let's keep it simple for now and turn words into integer ids

In [4]:
# Let's get the first example
raw_text = dataset['text'][0]
raw_text

"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!"

In [5]:
# The simplest tokenization we can do is splitting by white-space
tokens = raw_text.split(' ')
tokens

['Un-bleeping-believable!',
 'Meg',
 'Ryan',
 "doesn't",
 'even',
 'look',
 'her',
 'usual',
 'pert',
 'lovable',
 'self',
 'in',
 'this,',
 'which',
 'normally',
 'makes',
 'me',
 'forgive',
 'her',
 'shallow',
 'ticky',
 'acting',
 'schtick.',
 'Hard',
 'to',
 'believe',
 'she',
 'was',
 'the',
 'producer',
 'on',
 'this',
 'dog.',
 'Plus',
 'Kevin',
 'Kline:',
 'what',
 'kind',
 'of',
 'suicide',
 'trip',
 'has',
 'his',
 'career',
 'been',
 'on?',
 'Whoosh...',
 'Banzai!!!',
 'Finally',
 'this',
 'was',
 'directed',
 'by',
 'the',
 'guy',
 'who',
 'did',
 'Big',
 'Chill?',
 'Must',
 'be',
 'a',
 'replay',
 'of',
 'Jonestown',
 '-',
 'hollywood',
 'style.',
 'Wooofff!']

Ok, we have a list of words for the text. In order to turn it into integer ids, we need to build a map word -> id and viceversa (id -> word). This is what is called a vocabulary, and is an essential component of any deep learning NLP pipeline. Let's build our first vocabulary (just for this text sample):

In [6]:
unique_tokens = set(tokens) # remove repeated tokens 
unique_tokens

{'-',
 'Banzai!!!',
 'Big',
 'Chill?',
 'Finally',
 'Hard',
 'Jonestown',
 'Kevin',
 'Kline:',
 'Meg',
 'Must',
 'Plus',
 'Ryan',
 'Un-bleeping-believable!',
 'Whoosh...',
 'Wooofff!',
 'a',
 'acting',
 'be',
 'been',
 'believe',
 'by',
 'career',
 'did',
 'directed',
 "doesn't",
 'dog.',
 'even',
 'forgive',
 'guy',
 'has',
 'her',
 'his',
 'hollywood',
 'in',
 'kind',
 'look',
 'lovable',
 'makes',
 'me',
 'normally',
 'of',
 'on',
 'on?',
 'pert',
 'producer',
 'replay',
 'schtick.',
 'self',
 'shallow',
 'she',
 'style.',
 'suicide',
 'the',
 'this',
 'this,',
 'ticky',
 'to',
 'trip',
 'usual',
 'was',
 'what',
 'which',
 'who'}

In [7]:
word_to_id= {word: i for i, word in enumerate(tokens)} # turns words into ids
this_id = word_to_id['this']
this_id

49

In [8]:
id_to_word = {i: word for i, word in enumerate(tokens)} # turns ids into words
id_to_word[this_id]

'this'

Now we can use our 'vocab' to turn our text into a sequence of numbers:

In [9]:
numericalized_tokens = [word_to_id[w] for w in tokens]
numericalized_tokens

[0,
 1,
 2,
 3,
 4,
 5,
 18,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 50,
 53,
 29,
 30,
 49,
 32,
 33,
 34,
 35,
 36,
 37,
 63,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68]

In [10]:
# Now let's build a vocab for the whole dataset
all_tokens = []

In [11]:
for text in dataset['text']:
    all_tokens.extend(text.split(' '))
all_tokens[100:110]

['relatively',
 'cheery.',
 'There',
 'are',
 'no',
 'really',
 'superstars',
 'in',
 'the',
 'cast,']

In [12]:
len(all_tokens)

247797

In [13]:
unique_all_tokens = set(all_tokens)
# Get the size of our vocab
len(unique_all_tokens)

36462

In [14]:
# Build the vocab
word_to_id = {word: i for i, word in enumerate(unique_all_tokens)}
id_to_word = {i: word for i, word in enumerate(unique_all_tokens)}
john_id = word_to_id['John']

In [15]:
id_to_word[john_id+1] # next word in the vocab

'spirit.'

Now we have a really simple vocab for numericalizing our training/validation data.

# Turning words into vectors
We have now ids for every word. Almost every neural net for NLP uses this integer ids to get a vector for the word (or character, or..) in the first layer. This is what is know a the Embedding layer. The embedding layer is basically a lookup table of size Vxd, where V is the size of the vocab and d the dimension of the embedding vector. Let's see how this works:


In [16]:
import torch
from torch.nn.modules import Embedding
vocab_size = len(word_to_id)
emb_dim = 50 
embedding_layer = Embedding(vocab_size, emb_dim)
embedding_layer

Embedding(36462, 50)

In [17]:
# Let's get the vector for our first word
v_0 = embedding_layer(torch.tensor(0)) # The network only understands torch.tensor objects
v_0

tensor([ 1.1793, -1.6524, -0.8027, -0.5258, -0.7365,  0.1712,  0.6045, -0.4328,
         1.5618,  0.7033,  1.2411, -0.1594,  0.2673, -1.4940, -0.3764,  1.7147,
         0.6406,  0.7183,  0.6585, -0.2753, -1.6873,  2.4759, -0.1415,  0.7234,
        -0.8112, -2.1497, -1.5302, -0.8985, -0.0696, -0.2400,  0.6958, -0.8517,
        -0.5206, -1.1206, -0.8481, -0.9858, -2.1528,  0.3649, -0.7231,  0.7523,
         0.8982, -0.8649,  0.0142,  0.2519, -1.0422, -1.7638, -0.6293, -0.8947,
        -0.1100,  2.0343], grad_fn=<EmbeddingBackward>)

In [18]:
# Now let's try with our first full example
tokens = dataset['text'][0].split(' ')
numericalized_example = [word_to_id[w] for w in tokens]
'text of length {} tokens'.format(len(numericalized_example)), numericalized_example, tokens

('text of length 69 tokens',
 [18835,
  6409,
  14998,
  5652,
  28002,
  12392,
  17343,
  18368,
  31007,
  26791,
  16424,
  19000,
  292,
  34867,
  26388,
  18623,
  11167,
  20783,
  17343,
  15273,
  31565,
  23187,
  14630,
  29819,
  8429,
  36035,
  35439,
  29167,
  27079,
  4534,
  4418,
  15386,
  20719,
  6667,
  22267,
  8713,
  12333,
  30995,
  11174,
  762,
  14648,
  11460,
  32252,
  852,
  10052,
  22610,
  5242,
  11316,
  8995,
  15386,
  29167,
  25167,
  11764,
  27079,
  14127,
  20765,
  26110,
  14414,
  27368,
  1568,
  19893,
  33686,
  20920,
  11174,
  17895,
  24904,
  25560,
  5339,
  29996],
 ['Un-bleeping-believable!',
  'Meg',
  'Ryan',
  "doesn't",
  'even',
  'look',
  'her',
  'usual',
  'pert',
  'lovable',
  'self',
  'in',
  'this,',
  'which',
  'normally',
  'makes',
  'me',
  'forgive',
  'her',
  'shallow',
  'ticky',
  'acting',
  'schtick.',
  'Hard',
  'to',
  'believe',
  'she',
  'was',
  'the',
  'producer',
  'on',
  'this',
  'dog.

In [19]:
v_example = embedding_layer(torch.tensor(numericalized_example))
v_example # A matrix of the vectors corresponding to each of the 69 tokens

tensor([[-1.4529, -0.6323,  0.0181,  ...,  0.4517,  0.7590, -0.0408],
        [-0.2956,  0.3399,  0.6163,  ...,  0.0127,  0.5018, -1.1568],
        [ 0.1918, -0.2975, -1.3507,  ...,  1.2147, -1.3360,  0.9147],
        ...,
        [ 0.5374,  1.2180,  0.2631,  ...,  0.8405,  0.1004, -0.9676],
        [-0.2552, -1.2703, -0.0468,  ...,  1.2080,  0.4211, -1.3870],
        [-1.1998, -1.6173, -0.1548,  ...,  0.5007,  1.3911, -0.5079]],
       grad_fn=<EmbeddingBackward>)

In [20]:
v_example[0] # the embedding vector of the first token

tensor([-1.4529, -0.6323,  0.0181,  0.6775, -0.3333,  1.5223,  0.4538,  1.4436,
         1.0790, -0.0183, -1.1441, -0.9111, -0.3734, -0.0956,  0.5664,  0.6340,
        -2.6262, -1.6272, -1.6452,  0.9572, -1.3275, -0.3326, -0.4995, -1.5917,
        -0.1082,  0.8988,  0.5978,  0.5058,  0.0217, -0.0409, -0.2384,  0.1086,
         1.3744,  1.4670,  0.4075, -0.6104, -0.5774, -0.6275, -0.7707, -0.7730,
         0.1863, -0.9896,  0.1855,  1.5590,  0.1967,  1.6219, -1.6447,  0.4517,
         0.7590, -0.0408], grad_fn=<SelectBackward>)

Good job! Now we have turn text into 'dense' real-valued vectors! 
Now, let's try to generalize this a little bit.

But first, let's try our vocab on text outside the IMDB sample dataset.


In [21]:
my_movie_review_text = 'Climax from Gaspar Noé is a shockingly beatiful movie'.split(' ')
numericalized_movie_review = [word_to_id[w] for w in my_movie_review_text]

KeyError: 'Climax'

## Out of vocabulary words
What happened?

'Climax' is what it's called an out of vocabulary word (or oov, unk..). This is an important thing to deal with when working with supervised learning for NLP, as our model is expected to work with text not seen during training, validation or test. The simplest way to deal with this is to add a special token to our vocabulary which will be assigned to every unknown word. 

But for this let's generalize a little bit our vocabulary functionality.



In [22]:
class Vocab:
    def __init__(self, unk_symbol='<unk>', is_label=False):
        self.size = 0
        self.word_to_id = {}
        self.id_to_word = {}
        # you will understand this later
        if not is_label:
            self.unk_symbol = unk_symbol
            self.unk_id = self.add_word(unk_symbol)
    def add_word(self, w):
        if w not in self.word_to_id:
            self.word_to_id[w] = self.size
            self.id_to_word[self.size] = w
            self.size += 1
        return self.size - 1
    def to_id(self, w):
       return self.word_to_id[w] if w in self.word_to_id else self.unk_id
    def to_word(self, id):
       return self.id_to_word[id] if id in self.id_to_word else self.unk_symbol  
    def __len__(self):
        return self.size
vocab = Vocab()
vocab.to_id('Climax')

0

In [23]:
len(vocab)

1

In [24]:
vocab.add_word('Climax')

1

In [25]:
# Now, let's try to build the vocab for the full dataset
full_vocab = Vocab()
for text in dataset['text']:
    for w in text.split(' '):
        full_vocab.add_word(w)
len(full_vocab) # We should get our previous lenght + 1 (for the unk token) = 36463
    

36463

In [26]:
# Finally let's try on our previous unseen example
my_movie_review_text

['Climax',
 'from',
 'Gaspar',
 'Noé',
 'is',
 'a',
 'shockingly',
 'beatiful',
 'movie']

In [27]:
numericalized_movie_review = [full_vocab.to_id(w) for w in my_movie_review_text]
numericalized_movie_review

[0, 282, 0, 0, 66, 58, 25416, 0, 269]

In [28]:
numericalized_movie_review
[full_vocab.to_word(i) for i in numericalized_movie_review]

['<unk>', 'from', '<unk>', '<unk>', 'is', 'a', 'shockingly', '<unk>', 'movie']

In [29]:
# We got three unknown words, one of them a misspeling of beautiful
# Lets try with the right spelling
my_movie_review_text = 'Climax from Gaspar Noé is a shockingly beautiful movie'.split(' ')
numericalized_movie_review = [full_vocab.to_id(w) for w in my_movie_review_text]
numericalized_movie_review

[0, 282, 0, 0, 66, 58, 25416, 1534, 269]

In [31]:
full_vocab.to_word(1534)

'beautiful'

Now we have a working Vocab functionality with a simple tokenization mechanism (split by empty spaces), but what if we wanted a more general tokenizer with functions such as lowercasing, normalization, etc.?

Let's try to generalize this a little

In [32]:
class Tokenizer:
    def __init__(self, lowercase=False):
        self.lowercase = lowercase
    def __call__(self, text):
        return [w.lower() if self.lowercase else w for w in text.split(' ')]
my_tokenizer = Tokenizer(lowercase=True)
        

In [33]:
my_tokenizer('Climax is a horrible movie with nice music')

['climax', 'is', 'a', 'horrible', 'movie', 'with', 'nice', 'music']

## Exercise 1
Now please build a new vocab by tokenizing the full dataset with lowercased words:

In [34]:
# Now, let's try to build the vocab for the full dataset
lowercased_vocab = Vocab() 
for text in dataset['text']:
    # your code here
len(lowercased_vocab) # We should get a smaller vocab

IndentationError: expected an indented block (<ipython-input-34-c29e75453613>, line 5)

## Exercise 2
We have been focusing on text, but what about labels? Labels are frequently also text, like in our case, where we have `positive`and `negative`. Neural nets don't understand text, so what should we do? We need to turn them into numbers. Good news is we can reuse our previous vocab to do this!

Please create the vocab for labels. In this case we do not want the vocab to contain an unkwnow label, so we will use the is_label parameter

In [35]:
labels_vocab = Vocab(is_label=True)
# Create the labels vocab

# labels_vocab.word_to_id # check the generated vocab

Congrats! We have now almost everything we need to start training our network!